<a href="https://colab.research.google.com/github/UdaraChamidu/Generative-AI/blob/main/Text_Splitters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the necessary packages
!pip install langchain -qU
!pip install langchain-community -qU
!pip install unstructured -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.6/180.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 4.8 MB/s eta 0:00:00


# Load data

In [3]:
from langchain_community.document_loaders import DirectoryLoader

# Initialize the DirectoryLoader with the path to the directory for text files
loader = DirectoryLoader("sample_data", glob="**/*.txt")

# Load the text data from the directory
dataset = loader.load()

for data in dataset:
  print("------------------------")
  print(data.page_content)
  print(data.metadata)

------------------------
CodePRO LK is an educational platform offering a wide range of online courses primarily focused on programming and machine learning. All courses are provided for free and are delivered in Sinhala, catering specifically to the Sri Lankan audience. The platform includes various learning materials such as videos, quizzes, and assignments to enhance the learning experience​​.

Some of the featured courses on CodePRO LK include "Python GUI – Tkinter," "Machine Learning Project – Sentiment Analysis," and "Data Structures and Algorithms." These courses range from beginner to intermediate levels, making them accessible to a broad spectrum of learners​.

In addition to the courses, CodePRO LK also offers projects and resources to support learners in applying their knowledge practically. The platform emphasizes providing high-quality education to empower the next generation of Sri Lankans in the tech field​​.
{'source': 'sample_data/text2.txt'}
------------------------
L

In [4]:
# Calculate the token count of each document in the dataset using a character level tokenizer
token_counts = [len(data.page_content) for data in dataset]

print(token_counts)

# here, token=characters

[912, 1784]


# check openAI gpt 3.5 used tokenizer

In [6]:
# Install the tiktoken package for tokenization
!pip install tiktoken -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.2 MB/s eta 0:00:00


In [7]:
import tiktoken

tokenizer_model = tiktoken.encoding_for_model('gpt-3.5-turbo')
print(tokenizer_model)

<Encoding 'cl100k_base'>


In [8]:
# Get the encoding for the tokenizer
tokenizer = tiktoken.get_encoding('cl100k_base')

# Create a function to calculate the length of text in tokens using tiktoken
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    token_length = len(tokens)
    return token_length

In [9]:
# Calculate the token count of each document in the dataset using tiktoken
token_counts = [tiktoken_len(data.page_content) for data in dataset]

print(token_counts)

# according to chatgpt terbo 3,5 model tokenizer, provided txt files have differnt number of tokens.

[168, 286]


# recursive Character Splitter

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter with a chunk size of 150 and no overlap, using character length function
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    length_function=len,  # nummber of charactors
    separators=['\n\n', '\n', ' ', '']
)

In [19]:
# Split the text of the second document in the dataset into chunks
chunks = text_splitter.split_text(dataset[0].page_content)
# dataset[0] for 1 st document
len(chunks)

7

In [20]:
print(chunks[0]) # first chunk
print(len(chunks[0])) # number of tokens
print(chunks[1])
print(len(chunks[1]))

CodePRO LK is an educational platform offering a wide range of online courses primarily focused on programming and machine learning. All courses are
148
provided for free and are delivered in Sinhala, catering specifically to the Sri Lankan audience. The platform includes various learning materials
146


In [22]:
# Reinitialize the text splitter with a chunk size of 150 and an overlap of 20, using character length function
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
    length_function=len,
    separators=['\n\n', '\n', ' ', '']
)

In [24]:
# Split the text of the second document in the dataset into chunks
chunks = text_splitter.split_text(dataset[0].page_content)

len(chunks)

7

In [25]:
print(chunks[0])
print(len(chunks[0]))
print(chunks[1])
print(len(chunks[1]))

CodePRO LK is an educational platform offering a wide range of online courses primarily focused on programming and machine learning. All courses are
148
All courses are provided for free and are delivered in Sinhala, catering specifically to the Sri Lankan audience. The platform includes various
143


In [26]:
# Reinitialize the text splitter with a chunk size of 150 and an overlap of 20, using tiktoken length function
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=['\n\n', '\n', ' ', '']
)

In [29]:
# Split the text of the second document in the dataset into chunks
chunks = text_splitter.split_text(dataset[0].page_content)

len(chunks)

2

In [30]:
print(chunks[0])
print(tiktoken_len(chunks[0]))
print(chunks[1])
print(tiktoken_len(chunks[1]))

CodePRO LK is an educational platform offering a wide range of online courses primarily focused on programming and machine learning. All courses are provided for free and are delivered in Sinhala, catering specifically to the Sri Lankan audience. The platform includes various learning materials such as videos, quizzes, and assignments to enhance the learning experience​​.

Some of the featured courses on CodePRO LK include "Python GUI – Tkinter," "Machine Learning Project – Sentiment Analysis," and "Data Structures and Algorithms." These courses range from beginner to intermediate levels, making them accessible to a broad spectrum of learners​.
121
In addition to the courses, CodePRO LK also offers projects and resources to support learners in applying their knowledge practically. The platform emphasizes providing high-quality education to empower the next generation of Sri Lankans in the tech field​​.
47
